In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Exploratory Data Analysis

Importing modules and getting the data

In [ ]:
# Core Modules
import pandas as pd
import numpy as np

# Basic modules for data visualization
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

In [ ]:
# Load data into a pandas DataFrame from given filepath
train_path="../input/house-prices-advanced-regression-techniques/train.csv"
test_path="../input/house-prices-advanced-regression-techniques/test.csv"

train=pd.read_csv(train_path)
test=pd.read_csv(test_path)

housing = train

In [ ]:
# Getting column names of the dataframe
housing.columns

In [ ]:
# Getting number of features and observations
housing.shape

There are 1460 training examples in the dataset and 81 attributes.
One attribute is SalePrice which is the target variable and another the house Id, we can then say that we have 79 attributes that we can use for the analysis.

In [ ]:
# Exploring the top of the dataframe
housing.head()

In [ ]:
# I drop the column Id because it is not a useful attribute, and I check the description of the data
housing = housing.drop("Id", axis=1)
housing.info()

We have a mix of numerical and categorical variables. Several variebles have null values.

In [ ]:
# List of numerical attributes
num = housing.select_dtypes(exclude=['object']).columns
print(num)
print(len(num))

In [ ]:
# List of categorical attributes
cat = housing.select_dtypes(exclude=['number']).columns
print(cat)
print(len(cat))

There are 37 numerical and 43 categorical attributes in the dataset.
We get statistical summary for each of the two.

In [ ]:
#  Descriptive statistics with up to 2 decimals
# call transpose() for a better view of the results
housing.select_dtypes(exclude=['object']).describe().round(decimals=2).transpose()

In [ ]:
# Descriptive statistics of the categorical value
housing.select_dtypes(exclude=['number']).describe().round(decimals=2).transpose()

For the categorical attributes we get the number of unique values and the top value for each attribute.
For example 'HouseStyle' has 8 unique values, of which '1Story' is the most common with 726 occurrances.

In [ ]:
# Descriptive statistics summary of the target variable
housing['SalePrice'].describe()

In [ ]:
# Distribution plot of the target variable
sns.distplot(housing['SalePrice']);

From the plot we see that the variable is skewed, in fact the summary statistics indicate that 50% of the prices are below 163k (median) while the average price is 180k.

I apply a log transformation, by taking the log of the variable, in order to reduce the skew. This is important because I am going to apply linear regression which does not handle very good skewed data.

In [ ]:
# Log transformation to make the distribution more normal
sns.distplot(np.log(housing['SalePrice']))
plt.title('Distribuition of log-transformed SalesPrice')
plt.xlabel('log(SalesPrice)')
plt.show()

The plot looks now more normally distributed.

We can plot the distribution of the other 36 numerical attributes as well.
 

In [ ]:
# Filtering and create a copy of the numerical variables and excluding SalePrice
num_var = housing.select_dtypes(exclude = ['object']).drop(['SalePrice'], axis=1).copy()

# Plotting the variables
fig  = plt.figure(figsize=(14,20))
for i in range (len(num_var.columns)):
    fig.add_subplot(9, 4, i+1) # (nrows, ncols, index)
    #narrowing the bandwidth with kde_kws https://stackoverflow.com/a/61924418
    #hist= plot a (normed) histogram.
    #rug = draw a rugplot on the support axis.
    sns.distplot(num_var.iloc[:,i].dropna(), kde_kws={'bw': 0.1}, hist = False, rug = True)
    plt.xlabel(num_var.columns[i])
    
plt.tight_layout()
plt.show()

The plot show that the variables have different distributions and scales. 
Some Variables are very skewed such as 'LotArea'.

### Dealing with outliers

We can use box plots to spot outliers.

In [ ]:
# Plotting the variables on box plots
fig  = plt.figure(figsize=(10,15))
for i in range (len(num_var.columns)):
    fig.add_subplot(9, 4, i+1) # (nrows, ncols, index)
    sns.boxplot(y=num_var.iloc[:,i].dropna())
    
plt.tight_layout()
plt.show()

From a visual exploartion of the variables throught the box plots above, we can see that some variables have outliers, such as 'LotFrontage' (values above 200) and 'LotArea' (values above 150000).

### Correlations among numerical variables

We check whether some pairs of variables are highly correlated and remove them. Ideally we would like independent features.

We use the corr() method which returns the correlation for all pairs of variables in a range between -1 and 1, which 0 meaning no correlation. 

We visualize the correlation with a heatmap.

In [ ]:
# Correlation of numerical variables
corr = housing.corr()

# Using mask to get triangular correlation matrix
# https://www.kdnuggets.com/2019/07/annotated-heatmaps-correlation-matrix.html
f, ax = plt.subplots(figsize = (15,15))
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)]=True

sns.heatmap(corr, 
            mask=mask, 
#             cmap=sns.diverging_palette(220,10, as_cmap=True), 
            cmap = 'coolwarm',
            square = True, 
            ax=ax, 
            vmin = -1.0, 
            vmax =1.0, 
            linewidths = .5)

The heatmap shows some high correlations with the SalePrice (dark orange/red). We check the actual values of the correlations to confirm this.

In [ ]:
corr['SalePrice'].sort_values(ascending=False)

'OverallQual' and 'GrLivArea' seem highly correlated with 'SalePrice' because their values are close to 1. This means that when one of the two variables increase, SalePrice increase as well. 'PoolArea' and 'MoSold' have values close to 0 so they are not correlated to 'SalePrice'.

We create a scatterplot of these two highest correlated variables with 'SalePrice' to have a better visualisation of the relationship.

In [ ]:
var = ['SalePrice', 'OverallQual', 'GrLivArea']
sns.pairplot(housing[var])

Again we see that as the 'OverallQual' and 'GrLivArea' are highly correlated with 'SalePrice' : as one of the variables increase, SalePrice increases.

We can show this relationship with different plots as well.

In [ ]:
# Barplot
sns.barplot(housing.OverallQual, housing.SalePrice)

In [ ]:
# Boxplot
sns.boxplot(x=housing.OverallQual, y=housing.SalePrice)

These plots show again that as the overall quality of the house the price increase, but they also show that houses with higher quality have more variety in prices. 

We explore the relationship between SalePrice and YearBuilt, to check whether the age of the house influeces its price.

In [ ]:
# Creating a dataframe with only the variables of interest
age_price = pd.concat([housing['SalePrice'], housing['YearBuilt']], axis=1)
f, ax = plt.subplots(figsize=(32, 16))
fig = sns.boxplot(x='YearBuilt', y='SalePrice', data=age_price)
fig.axis(ymin=0, ymax=housing['SalePrice'].max());
plt.xticks(rotation=90);


We get the correlation for all the other variables as well. If two variables are hightly correlated we can consider removing one of them.

In [ ]:
# Only correlations above 0.5 and not auto-correlations, remove NaN and build a dictionary
# unstack to output a series
high_corrs = (corr[abs(corr) > 0.5][corr != 1.0]).unstack().dropna().to_dict()
unique_high_corrs = pd.DataFrame(list(set([(tuple(sorted(key)), 
                                            high_corrs[key])for key in high_corrs])), 
                                 columns=['attribute pair', 'correlation'])

# Sorted by absolute value
unique_high_corrs = unique_high_corrs.iloc[abs(unique_high_corrs['correlation']).argsort()[::-1]]

unique_high_corrs    

GarageCars and GarageArea are highly correlated. Because they also intuitively express similar features, we can drop one of them. Specifically it is better to drop GarageArea, since GarageCars is also highly correlated with SalePrice. 
We can also drop varibles that have very small correlation with SalePrice, such as MoSold, 3SsnPorch and BsmtFinSF2.

### Explore categorical attributes

We now do some exploratory data analysis on the categorical attributes

In [ ]:
cat

We check the relationship between KitchenQual and prices.

In [ ]:
# Boxplot
f, ax = plt.subplots(figsize = (10,6))
sns.boxplot(x=housing.KitchenQual, y=housing.SalePrice)
plt.show()

Houses with KitchenQual value Ex are more expensive (and also have slightly more variety in prices), while the Fa ones are the cheapest options.

We check now the style of the house and its relation with the price. We look at a box plot and the frequecy distribution of the values.

In [ ]:
f, ax = plt.subplots(figsize=(12,8))
sns.boxplot(y=housing.SalePrice, x=housing.HouseStyle)
plt.show()

# Count of categories within HouseStyle attribute
fig = plt.figure(figsize=(12, 4))
sns.countplot(x='HouseStyle', data=housing)
plt.ylabel('Frequency')
plt.show()


housing.HouseStyle.value_counts()

The most expensive option is the 2story and this option has also the largest variety in prices. 

Most of the houses are 1story.

We can plot the style of the house agaisnt the years to check whether the style has changed over the years.

In [ ]:
plt.scatter(housing.YearBuilt,housing.HouseStyle)

From the plot above we see that 1Story and 2Story are consistent over the years (a little more frequent in recent years) while 2.5Fin, 2.5Unf and 1.5Unf are all older houses and SFoyer and SLvl are all newer houses.

# Data Preprocessing

## Dealing With Missing Values

We get a sorted count of the missing values for all the attributes.

In [ ]:
housing.isnull().sum().sort_values(ascending=False)

Most of the categorical atttributes have missing values where the feature is not prensent, for example 'PoolQC' is NaN if the house does not have a pool.

So for the categorical values I will replace the missing values with None, except 'Electrical' (Electrical system), which will be filled with the mode.

Then for 'LotFrontage', which is the linear feet of street connected to property, I will fill the missing value with the median of the value for that attribute in the same neighborhood.

The numerical attributes will be filled with the zero.

The attribute 'Utilities' (Type of utilities available) will be dropped because not interesting for this model.


In [ ]:
# Imputing Missing Values

housing_fillNa = housing

# Categorical columns:
cat_cols_fill_none = ['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu',
                     'GarageCond', 'GarageQual', 'GarageFinish', 'GarageType',
                     'BsmtFinType2', 'BsmtExposure', 'BsmtFinType1', 'BsmtQual', 'BsmtCond',
                     'MasVnrType']

# Replace missing values for categorical columns with None
for cat in cat_cols_fill_none:
    housing_fillNa[cat] = housing_fillNa[cat].fillna("None")
    
# Use the mode value for Electrical
housing_fillNa['Electrical'] = housing_fillNa['Electrical'].fillna(housing_fillNa['Electrical']).mode()[0]
    
# Group by neighborhood and fill in missing value by the median LotFrontage of all the neighborhood
housing_fillNa['LotFrontage'] = housing_fillNa.groupby("Neighborhood")["LotFrontage"].transform(
    lambda x: x.fillna(x.median()))    

# Garage: GarageYrBlt, GarageArea, GarageCars and MasVnrArea these are numerical columns, replace with zero
for col in ['GarageYrBlt', 'GarageArea', 'GarageCars', 'MasVnrArea']:
    housing_fillNa[col] = housing_fillNa[col].fillna(int(0))

# There is no need of Utilities so let's just drop this column
housing_fillNa = housing_fillNa.drop(['Utilities'], axis=1)

# Get the count again to verify that we do not have any more missing values
housing_fillNa.isnull().apply(sum).max()

## Dealing with Outliers 

In order to remove the outliers, we remove the datapoints that are above the 99.9% quantile.

In [ ]:
# Numerocal attributes without NaN
num_attr = housing_fillNa.select_dtypes(exclude='object')

quantile = housing_fillNa.quantile(.999)

for col in num_attr.columns:
    housing_fillNa = housing_fillNa.drop(housing_fillNa[col][housing_fillNa[col]>quantile[col]].index)

housing_fillNa.info()

After removing the outliers, we are now left with 1422 rows.

## Dealing with Correlated Attributes

We remove the highly-correlated features, which we have identified previously. As for GarageArea and GarageCars, they are highly-correlated, however we will drop GarageArea because GarageCars is more correlated with price than area.

In [ ]:
#### Remove highly correlated features as identified for excluding in prev scatter plots & corr values
attributes_drop = ['MiscVal', 'MoSold', 'YrSold', 'BsmtFinSF2','BsmtHalfBath','MSSubClass',
                   'GarageArea', 'GarageYrBlt', '3SsnPorch']

housing_fillNa = housing_fillNa.drop(attributes_drop, axis=1)

## Handle Text And Categorical Attributes

We convert categorical number from text to number with one-hot encoding, where the value of the attribute will be 1 if it is present, and 0 if not.

In [ ]:
# One-hot encoding of categorical attributes
from sklearn.preprocessing import OneHotEncoder
cat_encoder = OneHotEncoder()
housing_fillNa_1hot = cat_encoder.fit_transform(housing_fillNa)
housing_fillNa_1hot

After this process, we have 7333 attributes, while before we had only 70.